# Simple HTML Study

Here we will get familiar with the syntax of the reVISitPy package. To do this, we'll recreate the config for the simple html study [here](https://github.com/revisit-studies/study/blob/main/public/demo-html-input/config.json). We'll start by first creating the study metadata and the UI configuration.

In [1]:
import revisitpy as rvt

study_metadata = rvt.studyMetadata(
    title='Passing Data From reVISit to HTML and back',
    version='pilot',
    authors=[
        'The reVISit Team',
        'reVISitPy'
    ],
    date='2025-01-20',
    description="A demo of how to pass visualization data from reVISit to HTML and send user answers from HTML back to the reVISit.",
    organizations=[
        "University of Utah",
        "WPI",
        "University of Toronto"
    ]
)

ui_config = rvt.uiConfig(
    contactEmail="contact@revisit.dev",
    helpTextPath="demo-html-input/assets/help.md",
    logoPath="revisitAssets/revisitLogoSquare.svg",
    withProgressBar=True,
    autoDownloadStudy=False,
    autoDownloadTime=5000,
    sidebar=False
)

In [ ]:
response_one = rvt.response(
    id='barChart',
    prompt='Your selected answer:',
    location='belowStimulus',
    # type='reactive'
    type='iframe'
)

base_component = rvt.component(
    component_name__='bar-chart',
    type='website',
    response=[response_one],
    path="demo-html-input/assets/bar-chart-interaction.html",
    instructionLocation='aboveStimulus'
)

comp_one = rvt.component(
    base__=base_component,
    component_name='bar-chart-1',
    description="A trial for the user to click the largest bar",
    instruction='Click on the largest bar',
    parameters={
        "barData": [
            0.32,
            0.01,
            1.2,
            1.3,
            0.82,
            0.4,
            0.3
        ]
    }
)

comp_two = rvt.component(
    base__=base_component,
    component_name='bar-chart-2',
    description="A trial for the user to click the smallest bar",
    instruction='Click on the smallest bar',
    parameters={
        "barData": [
            1.2,
            1.2,
            1.2,
            1.3,
            0.82,
            0.4,
            0.3
        ]
    }
)